In [ ]:
import folium
import geopandas
import feature_download
import os
from folium.plugins import GroupedLayerControl
from folium_glify_layer import GlifyLayer, Popup, Tooltip
import geojson
import json
import shapely
# Load the local geojson AOI file and get bbox in Albers projection
BASE_DIR = os.getcwd()
lup_aoi = os.path.join(BASE_DIR, '..', 'geojson', 'aoi.geojson')

# Initialize feature downloader
wfs = feature_download.WFS_downloader()

# Get bbox from AOI
aoi = geopandas.read_file(lup_aoi)
bbox_albers = wfs.create_bbox(aoi)
aoi = aoi.to_crs(4326)

# Define WFS getter
def wfs_getter(layer, query=None, fields=None, bbox=None):
    wfs_layer = wfs.get_data(layer, query=query, fields=fields, bbox=bbox)
    if 'GEOMETRY' in wfs_layer.columns:
        wfs_layer.set_geometry('GEOMETRY', inplace=True)
    elif 'SHAPE' in wfs_layer.columns:
        wfs_layer.set_geometry('SHAPE', inplace=True)
    wfs_layer = wfs_layer.set_crs('epsg:3005').to_crs(4326)
    return wfs_layer

def return_geoj (layer, query=None, fields=None, bbox=None):
    wfs_layer= wfs.wfs_query(layer,query=None, fields=None, bbox=None)
    collection = geojson.FeatureCollection(features=wfs_layer)
    
    return collection 
    # return wfs_layer



# for center starting point
lon, lat= -128.867888, 55.781113
# for controlled extent 
min_lon, max_lon= -127.120663, -127.120663
min_lat, max_lat= 54.658317, 56.880993
zoom_start = 7
basemap='OpenStreetMap'


#call folium map object
m = folium.Map(
    max_bounds=False,
    location=[lat, lon],
    prefer_canvas=True,
    tiles=basemap,
    zoom_start=zoom_start,
    control_scale=True
)

In [ ]:
legal_g_hawk_nesting= wfs_getter('WHSE_LAND_USE_PLANNING.RMP_PLAN_LEGAL_POLY_SVW', query="""LEGAL_FEAT_OBJECTIVE = 'Goshawk Nesting/Post-Fledging Habitat' And STRGC_LAND_RSRCE_PLAN_NAME IN ('Cranberry Sustainable Resource Management Plan', 'Nass South Sustainable Resource Management Plan')""", fields=['LEGAL_FEAT_ID','STRGC_LAND_RSRCE_PLAN_NAME', 'LEGAL_FEAT_OBJECTIVE', 'LEGALIZATION_DATE', 'ENABLING_DOCUMENT_TITLE', 'ENABLING_DOCUMENT_URL', 'RSRCE_PLAN_METADATA_LINK','GEOMETRY','OBJECTID'],bbox=bbox_albers)
#GLIFY only takes polygon, line or point not multi polygon
legal_g_hawk_nesting=legal_g_hawk_nesting.explode()

legal_g_hawk_nesting.loc[:, 'geometry'] = legal_g_hawk_nesting['geometry'].apply(lambda geom: geom if geom is None else shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=5)))




In [3]:
#geojson layer
# folium.GeoJson(legal_g_hawk_nesting, 
#             name='legal_g_hawk_nesting',
#             highlight_function=lambda x: {'fillOpacity': 0.8},
#             popup=legal_g_hawk_nesting,
#             zoom_on_click=True,).add_to(m)

In [4]:
legal_j=legal_g_hawk_nesting.to_json()
legal_j = json.loads(legal_g_hawk_nesting.to_json())

In [5]:
init_scale_function = """\
function(){
    return function(index){
        return { r: 0, g: 255, b: 0, a: 0.5 }; // Static green color with 50% transparency
    };
}
"""

color_feature_function = """\
function(index, feature) {
    return {
      r: 0,
      g: 255, // Full green for fill
      b: 0,
      a: 0.5  // 50% opacity for transparent fill
    };
}
"""
popup = Popup({
    'fields': ['LEGAL_FEAT_ID','STRGC_LAND_RSRCE_PLAN_NAME', 'LEGAL_FEAT_OBJECTIVE', 'LEGALIZATION_DATE', 'ENABLING_DOCUMENT_TITLE', 'ENABLING_DOCUMENT_URL', 'RSRCE_PLAN_METADATA_LINK'],
    'aliases': ['LEGAL_FEAT_ID','STRGC_LAND_RSRCE_PLAN_NAME', 'LEGAL_FEAT_OBJECTIVE', 'LEGALIZATION_DATE', 'ENABLING_DOCUMENT_TITLE', 'ENABLING_DOCUMENT_URL', 'RSRCE_PLAN_METADATA_LINK'],
    'labels': True
})

In [6]:
#Glify layer does not accept multiploygons, etc.
layer = GlifyLayer(
    feature_collections={'shapes': legal_j},
    glify_options={
        'border': True,
        'opacity': 0.5,
        # 'color': "{ r: 0, g: 128, b: 0, a: 1 }",
        
    },
    color_feature_function=color_feature_function,
    init_scale_function=init_scale_function,  # Add this line
    popup=popup, 
    
)

In [ ]:
layer.add_to(m)

In [ ]:
folium.LayerControl().add_to(m)

In [ ]:
m